In [1]:
# Importing Packages
        
import pandas as pd     
import numpy as np
from datetime import date, timedelta
import re
from IPython.display import display
import pyodbc
# ---------
from tweepy import OAuthHandler
import tweepy
import pandas as pd
import string
import os
import time

In [2]:
# API Keys

CONSUMER_KEY    = '9qH6416zLY2GENYQzvkeFyP2o'
CONSUMER_SECRET = '8XOo0ENIQy4HjX5jXC1xSdfj7jiOqubesglT29QsKkmHaDtEeh'

ACCESS_TOKEN  = '1219722321289777164-hlhI8xizj3M39DU6VdVhsPyQuAWbgF'
ACCESS_SECRET = 'l2WyzCqaaUfadV9P4N53zjJ3OOvqUb0IxNskk2GyOAiMM'

In [3]:
# Create authentication

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
key_words = [ 'leak' , 'burst' , 'pressure', 'water','cut', 'pipe',
              'fault', 'broke', 'damage', 'toilet', 'sanit',
              'waste', 'complain', 'supply','service','deliver',
              'out', 'no', 'reference','report' ,'complain',
              'install','low','repair']

In [204]:
def TweetCleaner(tweet):
    tweet = str(tweet.lower()) 
    tweet = re.sub('@[\w]*','',tweet) 
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
#     tweet = re.sub(r'#\w*', '', tweet) might put issue in hashtag
    tweet = re.sub(r"[,.;':@#?!\&/$]+\ *", ' ', tweet)
    tweet = re.sub(r"U+FFFD ", ' ', tweet)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet = tweet.lstrip(' ')
    tweet = ' '.join([i for i in tweet.split() if any(list_word in i for list_word in key_words)])
    if len(tweet)==0:
        tweet = np.nan
    return tweet

In [205]:
def scraptweets(districts, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'user_location',
                                        'geo_location','message','keywords',
                                        'hashtags','tweet_id','timestamp'])
    tweet_list = []
    for i in range(0, numRuns):
        for district in districts:
            tweets = tweepy.Cursor(api.search, q=district, lang="en", since=date_since, tweet_mode='extended').items(numTweets)
            district_list = [tweet for tweet in tweets]
            tweet_list += district_list
# Begin scraping the tweets individually:
    noTweets = 0
    for tweet in tweet_list:
        username = tweet.user.screen_name
        if tweet.user.location:
            user_location = tweet.user.location
        else:
            user_location = np.nan
        if tweet.place:
            geo_location = tweet.place.full_name
        else:
            geo_location = np.nan
        hashtags = ','.join([i['text'] for i in tweet.entities['hashtags']])
        if len(hashtags)==0:
            hashtags=np.nan
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            text = tweet.full_text
        keywords = TweetCleaner(text)
        tweet_id = tweet.id_str
        timestamp = tweet.created_at
        ith_tweet = [username,user_location,geo_location, text,keywords,hashtags,tweet_id,timestamp]
# Append to dataframe - db_tweets
        db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
        noTweets += 1
#     db_tweets.replace(np.nan,'',inplace=True)
    return(db_tweets)
#     time.sleep(920) #15 minute sleep time

In [255]:
# district_user_names = ['@alfrednzoDM','@AmajubaDM','@OfficialBCMM',
#                         '@CWDM2','@EhlanzeniM','@FezileDabiDM',
#                         '@GardenRoute_DM','@GertSibandeDM','@DistrictIlembe',
#                         '@Cetshwayo_dm','@LejweleputswaM','@MangaungCity',
#                         '@NkangalaDM','@SedibengDM','@UguDM','@UmkhanyakudeM',
#                         '@OkhahlambaLM','@VhembeDM','@RandLocal','@EmfuleniLM']
district_user_names = ['@thameswater']
date_since = date.today() - timedelta(days=2) #Searches up to past 2 days
numTweets = 100
numRuns = 1

In [256]:
new_df = scraptweets(district_user_names, date_since, numTweets, numRuns)

In [277]:
def add_to_DB(input_df):
    df = input_df.copy()
    from sqlalchemy import create_engine
    SERVER = 'LAPTOP-1SDCE2AE\SQLEXPRESS'
    DATABASE = 'WRC_DB'
    DRIVER = 'SQL Server Native Client 11.0'
    DATABASE_CONN = f'mssql://{SERVER}/{DATABASE}?trusted_connection=yes&driver={DRIVER}'
    engine = create_engine(DATABASE_CONN)
    connection = engine.connect()
    # Initialize large table if it doesnt exist:
    if not engine.dialect.has_table(engine, 'DB_Large'):
        df.to_sql('DB_Large',con=engine,index=False,chunksize=10000)
    # DEFINE FUNCTION TO FILTER AND CLASSIFY ISSUES
    # ALGORITHM TO INPUT SMALL TABLE HERE.. AS DF HAS TO BE FILTERED BY MODEL
    old_data = pd.read_sql_query('SELECT tweet_id FROM DB_Large',connection).tweet_id.tolist()
    to_drop = []
    for i,row in df.iterrows():
        if row['tweet_id'] in old_data:
            to_drop.append(i)
    df = df.drop(df.iloc[to_drop].index,axis=0)
    df.to_sql('DB_Large',con=engine,if_exists='append',index=False,chunksize=10000)
    return None

In [278]:
add_to_DB(new_df)